#  DAISY- the DAta-mIning SYnthetic-lethality-identification pipeline

Please cite: 

For Implementation: 

Our paper,

For DAISY algorithm: 

Jerby-Arnon, L., Pfetzer, N., Waldman, Y. Y., McGarry, L., James, D., Shanks, E., ... & Gottlieb, E. (2014). Predicting cancer-specific vulnerability via data-driven detection of synthetic lethality. Cell, 158(5), 1199-1209.

For CCLE Omics data:

Ghandi, M., Huang, F.W., Jané-Valbuena, J. et al. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019). https://doi.org/10.1038/s41586-019-1186-3

For CRISPR Data: 

Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.


This notebook is a reimplementation of DAISY Synthetic Lethal Pair Prediction Algorithm

Please first run the table_creation notebook before runnnig the DAISY notebook. 

It consists 3 modules: 

1. SL candidate determination using gene co-expression
2. SL candidate determination using survival of fittest
3. SL candidate determination using CRISPR and ShRNA experiment


* The results from the three modules were then aggregated into one ranked list of candidate SL pairs


Input Parameters
* Cancer type 
* The genes whose SL partners are seeked


Input Data
* Gene expression data 
* Gene mutation data
* Copy number variation data
* Gene effect data (CRISPR)
* Gene Dependency scores data (shRNA)

Output
* Ranked list of candidate SL pairs
![../../figures/daisy_pipeline.png](attachment:dene.png)

In [1]:
reset ####TK#### not sure if it is necessary

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
pwd ####TK#### not sure if it is necessary

'C:\\Users\\salta\\Documents\\GitHub\\SL-Cloud\\DAISY_pipeline'

### 1. Import python libraries required
The required libraries are imported. 

In [4]:
from datetime import datetime
import sys
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
sys.path.append('../scripts/') ####TK#### need to add "scripts" directory in a parent directory 
from google.cloud import bigquery
import importlib
import pandas as pd
import DAISY_operations
importlib.reload(DAISY_operations)
from DAISY_operations import *
from helper_functions import *
from BIGQUERY_operations import *

In [117]:
import sys ####TK#### duplicated

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

### 2. Sign in Google Bigquery with the project id

Bigquery connection
Please replace syntheticlethality with your project name

In [5]:
project_id='syntheticlethality'
client = bigquery.Client(project_id)
#client = bigquery.Client(credentials=credentials, project=credentials.project_id)

!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=SUCDtf9LC1JXaHzqdqgeiUewgCFATJ&access_type=offline&code_challenge=cMruscI9frhqTZ7yFXvOL4V3SwBPYVWs9uthQq7qxwM&code_challenge_method=S256


You are now logged in as [tkim@systemsbiology.org].
Your current project is [isb-cgc-04-0010].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [7]:
%load_ext google.cloud.bigquery

In [8]:
%%bigquery tsg

UsageError: %%bigquery is a cell magic, but the cell body is empty.


### 3. Input genes of interest for the SL partner prediction

We will predict synthetic lethal partner genes for tumor suppressor genes as default.
The query will use a permission required big-query table for tumor suppressor genes.
To execute this query, you need to register for a new COSMIC account and to get a permission.
Please follow this link for the registration : https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/data/COSMIC_about.html.
If you want to test the genes of your interest, please skip this and add you own genes to the variable "input_genes".

In [10]:
query='''
SELECT Gene_Symbol
  FROM `isb-cgc.COSMIC_v90_grch38.Cancer_Gene_Census` 
 WHERE Role_in_Cancer LIKE '%TSG%'

INTERSECT DISTINCT

SELECT HGNC_gene_symbol
  FROM `syntheticlethality.gene_information.cancer_driver_genes`
 
'''
driver_tsg_genes = client.query(query).result().to_dataframe()

<br>
Conversion from Hugo Symbols into EntrezIDs 

In [13]:
input_genes = driver_tsg_genes["Gene_Symbol"].to_list()
input_entrez_ids = ConvertGene(client, input_genes, 'Gene', ['EntrezID'])
input_entrez_ids

,Gene,EntrezID
0,APC,324
1,ATM,472
2,ATR,545
3,B2M,567
4,CIC,23152
...,...,...
108,PRKAR1A,5573
109,SMARCA4,6597
110,SMARCB1,6598
111,TBL1XR1,79718


### 4. Prediction of synthetic lethal partners using different modules on DAISY


There are three modules for synthetic lethal pair inferences on DAISY : 1. Pairwise gene coexpression, 2. Genomic survival of the fittest. 3. shRNA or CRISPR based functional examination. You can get more information in the original paper : https://www.sciencedirect.com/science/article/pii/S0092867414009775.

In pairwise gene coexpression module and genomic survial of the fittest module, we will use PancancerAtlas and CCLE data.<br>
In functional examination module, we will use CRISPR and shRNA data. <br>

Python codes for each module are built in our internal library (../scripts/SL_library.py) which was already imported at the beginning. 


#### 4.0. Default parameters for DAISY, you can edit them

In [14]:
input_mutations = ['Nonsense_Mutation', 'Frame_Shift_Ins', 'Frame_Shift_Del'] # Three mutation types were chosed as default by DAISY.
percentile_threshold = 10
cn_threshold = -0.3 
cor_threshold = 0.5
p_threshold = 0.05
pval_correction = 'Bonferroni'

#### 4.1. Pairwise gene coexpression module

4.1.1. Pairwise gene coexpression module on PancancerAtlas.

In [15]:
coexp_pancancer = CoexpressionAnalysis(client, "PanCancerAtlas", input_entrez_ids['EntrezID'][0:1], cor_threshold, p_threshold, pval_correction) 

C:\Users\salta\miniconda3\lib\site-packages\statsmodels\stats\multitest.py:150: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
C:\Users\salta\miniconda3\lib\site-packages\statsmodels\stats\multitest.py:254: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
../scripts\DAISY_operations.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['PValue']=FDR


In [16]:
coexp_pancancer

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                               
APC           0                  324           APC                 146057   
              172                324           APC                   8411   
              173                324           APC                   4763   
              174                324           APC                  26046   
              175                324           APC                  91526   
...                              ...           ...                    ...   
              88                 324           APC                  58508   
              87                 324           APC                   7638   
              86                 324           APC                   9101   
              92                 324           APC                   6655   
              271                324           APC                  79088   

                  Gene_SL_Candidate  Correlation  PValue  
Gene_Inactive                                             
APC           0               TTBK2     0.712054     0.0  
              172              EEA1     0.535663     0.0  
              173               NF1     0.535011     0.0  
              174              LTN1     0.534041     0.0  
              175           ANKRD44     0.534037     0.0  
...                             ...          ...     ...  
              88              KMT2C     0.570013     0.0  
              87             ZNF221     0.570202     0.0  
              86               USP8     0.570952     0.0  
              92               SOS2     0.567321     0.0  
              271            ZNF426     0.500762     0.0  

[272 rows x 6 columns]

<br>
You can save the results into bigquery table.

In [17]:
CreateTable(client, coexp_pancancer, 'pipeline_results', 'DAISY_coexpression_pancancer_sl_pairs_trial', project_id, "") 

Table could not be created
Table description could not be updated


<br>
4.1.2. Pairwise gene coexpression module on CCLE data

In [18]:
coexp_CCLE=CoexpressionAnalysis(client, "CCLE", input_entrez_ids['EntrezID'][0:1], cor_threshold, p_threshold, pval_correction) 

C:\Users\salta\miniconda3\lib\site-packages\statsmodels\stats\multitest.py:150: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
C:\Users\salta\miniconda3\lib\site-packages\statsmodels\stats\multitest.py:254: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1
../scripts\DAISY_operations.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['PValue']=FDR


In [19]:
coexp_CCLE

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                               
APC           0                  324           APC                  64848   
              1                  324           APC                  51735   
              2                  324           APC                    373   
              3                  324           APC                  27125   
              4                  324           APC                   4090   
...                              ...           ...                    ...   
              137                324           APC                   5469   
              138                324           APC                  51164   
              139                324           APC                  23367   
              140                324           APC                  10746   
              141                324           APC                  55662   

                  Gene_SL_Candidate  Correlation         PValue  
Gene_Inactive                                                    
APC           0              YTHDC2     0.636174  1.200051e-144  
              1             RAPGEF6     0.624914  5.689873e-138  
              2              TRIM23     0.624118  1.647260e-137  
              3                AFF4     0.607897  2.183924e-128  
              4               SMAD5     0.599476  7.424228e-124  
...                             ...          ...            ...  
              137              MED1     0.501736   7.567374e-80  
              138             DCTN4     0.501515   9.181577e-80  
              139             LARP1     0.500949   1.506241e-79  
              140            MAP3K2     0.500912   1.554829e-79  
              141            HIF1AN     0.500437   2.353087e-79  

[142 rows x 6 columns]

Results are saved into bigquery table

In [128]:
CreateTable(client, coexp_CCLE, 'pipeline_results', 'DAISY_coexpression_CCLE_sl_pairs_trial', project_id, "")

1it [00:02,  2.38s/it]


Table created successfully


#### 4.2. Genomic survival of fittest module

4.2.1. Genomic survival of fittest module on CCLE data

In [24]:
sof_CCLE = SurvivalOfFittest(client, "CCLE", p_threshold, input_entrez_ids['EntrezID'][0:1], input_mutations, percentile_threshold, cn_threshold, pval_correction)


../scripts\DAISY_operations.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['PValue']=FDR


In [25]:
sof_CCLE

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                               
APC           0                  324           APC                 160857   
              1                  324           APC                  10129   
              2                  324           APC              100507099   
              3                  324           APC                 196549   
              4                  324           APC                   1997   
...                              ...           ...                    ...   
              155                324           APC                 341676   
              156                324           APC                   9724   
              158                324           APC              101926951   
              157                324           APC                  64881   
              159                324           APC                  22862   

                  Gene_SL_Candidate    PValue  
Gene_Inactive                                  
APC           0             CCDC122  0.003147  
              1                 FRY  0.004754  
              2             FRY-AS1  0.004754  
              3             EEF1DP3  0.006678  
              4                ELF1  0.006858  
...                             ...       ...  
              155              NEK5  0.047518  
              156            UTP14C  0.047518  
              158         LINC02339  0.047648  
              157            PCDH20  0.047648  
              159            FNDC3A  0.048796  

[160 rows x 5 columns]

Results are saved into bigquery table

In [131]:
CreateTable(client, sof_CCLE, 'pipeline_results', 'DAISY_sof_CCLE_sl_pairs_trial', project_id, "")

1it [00:02,  2.82s/it]


Table created successfully


<br>
4.2.2. Genomic survival of fittest module on PancancerAtlas

In [26]:
sof_pancancer = SurvivalOfFittest(client, "PanCancerAtlas", p_threshold, input_entrez_ids['EntrezID'][0:1], input_mutations, percentile_threshold, cn_threshold,pval_correction)


../scripts\DAISY_operations.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report['PValue']=FDR


In [27]:
sof_pancancer

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                                
APC           0                   324           APC              100130274   
              1021                324           APC                   4987   
              1020                324           APC                 154761   
              1019                324           APC                   1129   
              1018                324           APC                 414919   
...                               ...           ...                    ...   
              5006                324           APC                   7272   
              5007                324           APC                    318   
              5008                324           APC                 347240   
              5009                324           APC                 151234   
              5010                324           APC              100113379   

                   Gene_SL_Candidate    PValue  
Gene_Inactive                                   
APC           0              CCDC166  0.000000  
              1021             OPRL1  0.000000  
              1020         LOC154761  0.000000  
              1019             CHRM2  0.000000  
              1018           C8orf82  0.000000  
...                              ...       ...  
              5006               TTK  0.046663  
              5007             NUDT2  0.048370  
              5008             KIF24  0.048848  
              5009         SULT1C2P1  0.048999  
              5010         SNORD121A  0.049908  

[5011 rows x 5 columns]

Results are saved in bigquery table

In [134]:
CreateTable(client, sof_pancancer, 'pipeline_results', 'DAISY_sof_pancancer_sl_pairs_trial', project_id, "")

1it [00:03,  3.33s/it]


Table created successfully


#### 4.3. Functional examination inference module

4.3.1. CRISPR based functional examination inference module

In [29]:
crispr_result = FunctionalExamination(client, "CRISPR", p_threshold, input_entrez_ids['EntrezID'][0:1], percentile_threshold, cn_threshold, 'none')


In [30]:
crispr_result

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                                
APC           0                   324           APC                   6728   
              1                   324           APC                   6943   
              2                   324           APC                   7766   
              3                   324           APC                   8178   
              4                   324           APC                  66000   
...                               ...           ...                    ...   
              1061                324           APC                   1673   
              1062                324           APC                  84818   
              1059                324           APC                  26509   
              1060                324           APC                  23678   
              1063                324           APC                  65982   

                   Gene_SL_Candidate    PValue  
Gene_Inactive                                   
APC           0                SRP19  0.000001  
              1                TCF21  0.000044  
              2               ZNF223  0.000047  
              3                  ELL  0.000161  
              4              TMEM108  0.000239  
...                              ...       ...  
              1061            DEFB4A  0.049898  
              1062            IL17RC  0.049898  
              1059              MYOF  0.049898  
              1060              SGK3  0.049898  
              1063           ZSCAN18  0.049898  

[1064 rows x 5 columns]

Results are saved in bigquery table

In [137]:
CreateTable(client, crispr_result, 'pipeline_results', 'DAISY_func_ex_crispr_sl_pairs_trial', project_id, "")

1it [00:11, 11.31s/it]


Table created successfully


<br>
4.3.2. shRNA based functional examination inference module

In [31]:
siRNA_result = FunctionalExamination(client, "siRNA", p_threshold, input_entrez_ids['EntrezID'][0:1], percentile_threshold, cn_threshold, 'none')


In [32]:
siRNA_result

EntrezID_Inactive Gene_Inactive  EntrezID_SL_Candidate  \
Gene_Inactive                                                               
APC           0                  324           APC                  10890   
              1                  324           APC                  79981   
              2                  324           APC                  81857   
              3                  324           APC                  79582   
              4                  324           APC                   2778   
...                              ...           ...                    ...   
              982                324           APC                 162966   
              984                324           APC                 400002   
              980                324           APC                  63899   
              981                324           APC                   5368   
              985                324           APC                   6120   

                  Gene_SL_Candidate    PValue  
Gene_Inactive                                  
APC           0               RAB10  0.000065  
              1               FRMD1  0.000183  
              2               MED25  0.000204  
              3              SPAG16  0.000523  
              4                GNAS  0.000603  
...                             ...       ...  
              982            ZNF600  0.049828  
              984       CLEC12A-AS1  0.049828  
              980             NSUN3  0.049828  
              981              PNOC  0.049828  
              985               RPE  0.049928  

[986 rows x 5 columns]

Results are saved in bigquery table

In [140]:
CreateTable(client, siRNA_result, 'pipeline_results', 'DAISY_func_ex_siRNA_sl_pairs_trial', project_id, "")

1it [00:02,  2.95s/it]


Table created successfully


### 5. Integration of results

5.1. Integration of the pairwise Co-expression gene co-expression results on Pancancer and CCLE

In [33]:
coexpression_result = UnionResults([coexp_pancancer, coexp_CCLE])

C:\Users\salta\miniconda3\lib\site-packages\scipy\stats\stats.py:7661: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


In [34]:
coexpression_result

,EntrezID_Inactive,Gene_Inactive,EntrezID_SL_Candidate,Gene_SL_Candidate,Correlation_0,PValue_0,Correlation_1,PValue_1,PValue
0,324,APC,146057,TTBK2,0.712054,0.0,NaN,NaN,0.000000e+00
1,324,APC,8411,EEA1,0.535663,0.0,NaN,NaN,0.000000e+00
2,324,APC,4763,NF1,0.535011,0.0,NaN,NaN,0.000000e+00
3,324,APC,26046,LTN1,0.534041,0.0,NaN,NaN,0.000000e+00
4,324,APC,91526,ANKRD44,0.534037,0.0,NaN,NaN,0.000000e+00
...,...,...,...,...,...,...,...,...,...
347,324,APC,9202,ZMYM4,NaN,NaN,0.502055,5.723647e-80,5.723647e-80
348,324,APC,5469,MED1,NaN,NaN,0.501736,7.567374e-80,7.567374e-80
349,324,APC,23367,LARP1,NaN,NaN,0.500949,1.506241e-79,1.506241e-79
350,324,APC,10746,MAP3K2,NaN,NaN,0.500912,1.554829e-79,1.554829e-79


In [143]:
CreateTable(client, coexpression_result, 'DAISY_RESULTS', 'Coexpression_Union_Pancancer_CCLE_trial', project_id, "")

Table could not be created
Table description could not be updated


<br>
5.2. Integration of Survival of Fittest results on Pancancer and CCLE

In [35]:
sof_result = UnionResults([sof_CCLE, sof_pancancer])

C:\Users\salta\miniconda3\lib\site-packages\scipy\stats\stats.py:7661: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


In [36]:
sof_result

,EntrezID_Inactive,Gene_Inactive,EntrezID_SL_Candidate,Gene_SL_Candidate,PValue_0,PValue_1,PValue
0,324,APC,160857,CCDC122,0.003147,0.000000,0.000000
1,324,APC,10129,FRY,0.004754,0.000000,0.000000
2,324,APC,100507099,FRY-AS1,0.004754,0.000000,0.000000
3,324,APC,196549,EEF1DP3,0.006678,0.000000,0.000000
4,324,APC,1997,ELF1,0.006858,0.000000,0.000000
...,...,...,...,...,...,...,...
5036,324,APC,7272,TTK,NaN,0.046663,0.046663
5037,324,APC,318,NUDT2,NaN,0.048370,0.048370
5038,324,APC,347240,KIF24,NaN,0.048848,0.048848
5039,324,APC,151234,SULT1C2P1,NaN,0.048999,0.048999



Results are saved in bigquery table

In [146]:
CreateTable(client, sof_result, 'DAISY_RESULTS', 'SOF_Union_Pancancer_CCLE_trial', project_id)

TypeError: CreateTable() missing 1 required positional argument: 'table_desc'

<br>
5.3. Integration of shRNA and CRISPR based functional examination inference module.

In [37]:
functional_screening_result = UnionResults([crispr_result, siRNA_result])

In [ ]:
CreateTable(client, functional_screening_result, 'DAISY_RESULTS', 'FuncEx_Union_CRISPR_siRNA_trial', project_id, "")

<br>
5.4. Merging the results from all three inference procedures

In [38]:
all_merged_results = MergeResults([coexpression_result, sof_result, functional_screening_result])

C:\Users\salta\miniconda3\lib\site-packages\scipy\stats\stats.py:7661: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


In [39]:
all_merged_results

,EntrezID_Inactive,Gene_Inactive,EntrezID_SL_Candidate,Gene_SL_Candidate,PValue
0,324,APC,26060,APPL1,0.000000e+00
1,324,APC,9031,BAZ1B,1.582141e-159


Results are saved in bigquery tables

In [ ]:
CreateTable(client, all_merged_results, 'pipeline_results', 'DAISY_final_sl_pairs_trial', project_id, "")

Results are saved in excel file

In [ ]:
WriteToExcel("tsg_driver.results_trial.xlsx", [coexp_pancancer,  coexp_CCLE,  sof_CCLE, sof_pancancer,  crispr_result, siRNA_result,  coexpression_result,  sof_result, functional_screening_result, co_ex_func_merged_results,  co_ex_sof_merged_results, sof_func_merged_results, all_merged_results],["Co-exp_Pancancer",  "Co-exp_CCLE" , "SOF_CCLE",  "SOF_Pancancer",  "CRISPR", "siRNA" , "Coexp_Union",  "Sof_Union", "Func_Sc_Union", "Coexp_Func_Merged", "Coexp_Sof_Merged", "Sof_Fun_Merged", "All"])


In [ ]:
end_time= datetime.now()